In [ ]:
dataset_split

In [ ]:
最近写的几个ipynb都非常长，就容易卡。
——这个ipynb【只负责】：
将整体的数据集分成3个集合：train1，train2，test
共5个csv：3个集合各1个，2个train的y各1个

In [ ]:
数据集B
分割方式和数据集A一致

In [ ]:
分割标准：
以1-16数据 预测17-23某用户是否活跃
以8-23数据 预测24-30某用户是否活跃
以15-30数据 预测31-37某用户是否活跃

In [1]:
import numpy as np
import pandas as pd

In [2]:
user_activity_log = pd.read_csv('dataset_B_for_model_method/origin_dataset/user_activity_log.csv')
table123 = pd.read_csv('dataset_B_for_model_method/origin_dataset/table_merge_first_3.csv')

In [3]:
table123.head()

,user_id,register_day,register_type,device_type,app_launch_day,video_create_day,video_create_day_count
0,8,9.0,0.0,34.0,9.0,NaN,NaN
1,8,9.0,0.0,34.0,10.0,NaN,NaN
2,8,9.0,0.0,34.0,11.0,NaN,NaN
3,8,9.0,0.0,34.0,13.0,NaN,NaN
4,8,9.0,0.0,34.0,18.0,NaN,NaN


In [4]:
user_activity_log.head()

,user_id,activity_day,activity_page,video_id,author_id,action_type,activity_count
0,8,9,3,1955966,309219,0,2
1,8,9,0,1340816,12378,0,1
2,8,9,0,3488372,107008,0,1
3,8,9,0,1705402,37639,0,2
4,8,9,0,3710937,1312718,0,1


In [ ]:
说明：
采集信息的区间如上所述。
但是，在总和完所有的特征之后，还要把各自区间的注册user_id加进来——
因为存在这种情况：
在15到30号期间，这些user_id没有行为，登录，视频信息，
这些user_id在15号之前注册，可能有过活动，但是在15到30号没有活动。
这些user_id是要被加入当做一部分样本的——【这样才合理】

In [ ]:
开始提取x

In [ ]:
table123

In [5]:
table123_1_16 = table123[table123['app_launch_day'] < 17]
table123_8_23 = table123[table123['app_launch_day'] < 24]
table123_8_23 = table123_8_23[table123_8_23['app_launch_day'] > 7]
table123_15_30 = table123[table123['app_launch_day'] > 14]

In [6]:
table123_1_16.shape,table123_8_23.shape,table123_15_30.shape,table123.shape

((79630, 7), (133965, 7), (188810, 7), (252516, 7))

In [ ]:
关于数据集shape的说明：
很明显，最后一个table的条目数最多，180000,比70000多了很多了。
原因也很简单，从1号到30号，是不停有新用户注册的，因此登录条目数，自然会变多。
重要的是，时间区间3个dataset保持一致即可。

In [ ]:
table4

In [7]:
act_1_16 = user_activity_log[user_activity_log['activity_day'] < 17]
act_8_23 = user_activity_log[user_activity_log['activity_day'] < 24]
act_8_23 = act_8_23[act_8_23['activity_day'] > 7]
act_15_30 =  user_activity_log[user_activity_log['activity_day'] > 14]

In [8]:
act_1_16.shape,act_8_23.shape,act_15_30.shape,user_activity_log.shape

((5070537, 7), (9204266, 7), (13576708, 7), (17635062, 7))

In [ ]:
分别存储x

In [9]:
table123_1_16.to_csv('dataset_B_for_model_method/split_origin_dataset/table123_1_16.csv',index = False)
act_1_16.to_csv('dataset_B_for_model_method/split_origin_dataset/act_1_16.csv',index = False)
table123_8_23.to_csv('dataset_B_for_model_method/split_origin_dataset/table123_8_23.csv',index = False)
act_8_23.to_csv('dataset_B_for_model_method/split_origin_dataset/act_8_23.csv',index = False)
table123_15_30.to_csv('dataset_B_for_model_method/split_origin_dataset/table123_15_30.csv',index = False)
act_15_30.to_csv('dataset_B_for_model_method/split_origin_dataset/act_15_30.csv',index = False)

In [ ]:
设计y
——注意显然只有两个

In [ ]:
提取出在范围内的user名单（包括17-23号注册的用户）

In [10]:
temp = table123[table123['app_launch_day'] > 16]
temp = temp[temp['app_launch_day'] < 24]
temp = temp['user_id']
temp1 = set(list(temp))

In [11]:
temp = user_activity_log[user_activity_log['activity_day'] > 16]
temp = temp[temp['activity_day'] < 24]
temp = temp['user_id']
temp2 = set(list(temp))

In [ ]:
取两个数据集的交集 |

In [12]:
origin_17_23 = temp1 | temp2

In [13]:
origin_17_23 = list(origin_17_23)

In [14]:
len(list(temp1)),len(list(temp2)),len(origin_17_23)

(25841, 22625, 25842)

In [ ]:
上边获得的是17-23的所有有记录的用户，现在要提取出注册日期在1-16号的

In [15]:
user_1_16 = list(set(list(table123_1_16['user_id'])))
len(user_1_16)

23923

In [ ]:
取两者的交集 &

In [16]:
active_1_16 = set(user_1_16) & set(origin_17_23)

In [17]:
active_1_16 = list(active_1_16)

In [18]:
len(active_1_16),len(user_1_16),len(origin_17_23)

(12430, 23923, 25842)

In [ ]:
结论：12430/23923,近50%的活跃率
——和数据集A类似的比例

In [ ]:
接着是24-30的

In [19]:
temp = table123[table123['app_launch_day'] > 23]
temp = temp['user_id']
temp1 = set(list(temp))

In [20]:
temp = user_activity_log[user_activity_log['activity_day'] > 23]
temp = temp['user_id']
temp2 = set(list(temp))

In [21]:
origin_24_30 = temp1 | temp2
origin_24_30 = list(origin_24_30)
len(list(temp1)),len(list(temp2)),len(origin_24_30)

(32376, 27415, 32383)

In [22]:
user_8_23 = list(set(list(table123_8_23['user_id'])))
len(user_8_23)

33936

In [23]:
active_8_23 = set(user_8_23) & set(origin_24_30)
active_8_23 = list(active_8_23)
len(active_8_23),len(user_8_23),len(origin_24_30)

(17996, 33936, 32383)

In [ ]:
17996/33936

In [ ]:
存档y（活跃名单）

In [24]:
active_1_16 = pd.DataFrame(active_1_16)

In [25]:
active_1_16.columns = ['user_id']

In [26]:
active_1_16.head()

,user_id
0,8
1,1212424
2,1310730
3,819211
4,1376267


In [27]:
active_1_16.to_csv('dataset_B_for_model_method/split_origin_dataset/y_1_16.csv',index = False)

In [28]:
active_8_23 = pd.DataFrame(active_8_23)
active_8_23.columns = ['user_id']
active_8_23.head()

,user_id
0,327687
1,8
2,1212424
3,1310730
4,819211


In [29]:
active_8_23.to_csv('dataset_B_for_model_method/split_origin_dataset/y_8_23.csv',index = False)